# Face Mask Detection

In [1]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.special import softmax

In [2]:
# Loading Face Detection Model
fd_model = cv2.dnn.readNetFromCaffe('D:/tuansihan/06 Projects/08 Face Mask Detection System/models/deploy.prototxt.txt',
                                    'D:/tuansihan/06 Projects/08 Face Mask Detection System/models/res10_300x300_ssd_iter_140000_fp16.caffemodel')

In [3]:
# Loading Face Mask Recognition Model
model = tf.keras.models.load_model('face_mask_model/')

In [4]:
# Declaring Labels
labels = ['Covered', 'Covered', 'No Mask', 'Nose Uncovered']

In [5]:
# Colors for Labels

def getColor(label):

    if label == "Covered":
        color = (0, 255, 0)

    elif label == "No Mask":
        color = (0, 0, 255)

    elif label == "Nose Uncovered":
        color = (0, 255, 255)

    else:
        color = (255, 0, 255)

    return color

## Testing the Prediction

In [6]:
# Import Test Image
#img = cv2.imread('test-masked.jpg')

# cv2.imshow('Original Image', img)
# cv2.waitKey()
# cv2.destroyAllWindows()

### Face Detection

In [7]:
def facemask_prediction(img):
    
    image = img.copy()
    h, w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1, (300, 300),(104, 117, 123), swapRB = True)

    fd_model.setInput(blob)
    detection = fd_model.forward() # Get Detection

    for i in range(0, detection.shape[2]):
        confidence = detection[0, 0, i, 2]
        if confidence > 0.5:
            #print(confidence)
            box = detection[0, 0, i, 3:7] * np.array([w, h, w, h])
            box = box.astype(int)
            pt1 = (box[0], box[1]) # Diagonals
            pt2 = (box[2], box[3]) # Diagonals
            #cv2.rectangle(image, pt1, pt2,(0,255,0), 1) # Draw Rectangle

            # Crop Face
            face = image[box[1]:box[3], box[3]:box[2]]
            blob = cv2.dnn.blobFromImage(face, 1, (100, 100), (104, 117, 123), swapRB = True)
            compressed_blob = np.squeeze(blob).T
            rotated_blob = cv2.rotate(compressed_blob, cv2.ROTATE_90_CLOCKWISE)
            flipped_blob = cv2.flip(rotated_blob, 1)
            img_no_neg = np.maximum(flipped_blob, 0)
            norm_img = img_no_neg/flipped_blob.max()

            # Implementing CNN
            img_input = norm_img.reshape(1,100,100,3) # Reshape to 4D
            #print(img_input.shape)
            result = model.predict(img_input)
            #print(result)
            result = softmax(result)[0]
            conf_index = result.argmax() # get position with the highest value
            #print(conf_index)
            conf_score = result[conf_index]
            label = labels[conf_index]
            label_text = '{}: {:,.0f} %'.format(label, conf_score*100)
            #print(label_text)

            # Color Box
            color = getColor(label)
            cv2.rectangle(image, pt1, pt2, color, 1)
            cv2.putText(image, label_text, (pt1), cv2.FONT_HERSHEY_PLAIN, 2, color, 1)
        
        return image

In [8]:
#softmax(result)

In [9]:
# img = cv2.imread('test-masked.jpg')

# image = facemask_prediction(img)

# cv2.imshow('Prediction', image)
# cv2.waitKey()
# cv2.destroyAllWindows()

## Real-Time Detection

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frames = cap.read()
    if ret == False:
        break
        
    image = facemask_prediction(frames)
    cv2.imshow('Face Mask Prediction System', image)
    if cv2.waitKey(1) == 27:
        break
        
cap.release()
cv2.destroyAllWindows()